# 基础思路（baseline）

## 步骤1：更新或安装所需环境

:适用Colab。A100(40Gb)可跑通。L4（22Gb)、T4(16Gb)实例会出现Oom，需要24Gb显存。

https://huggingface.co/THUDM/glm-4-9b-chat

In [1]:
!pip install --upgrade transformers requests urllib3 tqdm pandas tiktoken
!apt update > /dev/null; apt install aria2 git-lfs axel -y > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 102.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
ERROR: pip's dependency 

###补充一个包###

In [ ]:
#pip install tiktoken 已补充上去

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.2 MB/s eta 0:00:00


## 步骤2：下载数据集

In [2]:
!axel -n 12 -a https://ai-contest-static.xfyun.cn/2024/%E5%A4%A7%E6%A8%A1%E5%9E%8B%E8%83%BD%E5%8A%9B%E8%AF%84%E6%B5%8B%EF%BC%9A%E4%B8%AD%E6%96%87%E6%88%90%E8%AF%AD%E9%87%8A%E4%B9%89%E4%B8%8E%E8%A7%A3%E6%9E%90%E6%8C%91%E6%88%98%E8%B5%9B/test_input.csv

Initializing download: https://ai-contest-static.xfyun.cn/2024/%E5%A4%A7%E6%A8%A1%E5%9E%8B%E8%83%BD%E5%8A%9B%E8%AF%84%E6%B5%8B%EF%BC%9A%E4%B8%AD%E6%96%87%E6%88%90%E8%AF%AD%E9%87%8A%E4%B9%89%E4%B8%8E%E8%A7%A3%E6%9E%90%E6%8C%91%E6%88%98%E8%B5%9B/test_input.csv
File size: 131.332 Kilobyte(s) (134484 bytes)
Opening output file test_input.csv
Starting download

Can't setup alternate output. Deactivating.
..... .......... .......... .......... ..........  [  44.8KB/s]
[ 38%]  .......... .......... .......... .......... ..........  [  73.1KB/s]
[ 76%]  .......... .......... .......... .

Downloaded 131.332 Kilobyte(s) in 1 second(s). (92.31 KB/s)


## 步骤3：构建模型（使用GLM-4-9B-Chat模型）

---



 设置可见显存：在python文件外部加入CUDA_VISIBLE_DEVICES=1,2，指定特定的显卡。
 重要！单台机器如果有的显卡占满，有的显卡空着。加入device_map="auto"，仍然会出现爆显存的情况。主要因为device_map="auto"会将模型分配到满显存的卡上。

解决：通过CUDA_VISIBLE_DEVICES=1,2指定显存充足的卡，避开显存不足的卡。

多卡时：命令行运行：CUDA_VISIBLE_DEVICES=1,2 python run_infer.py

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

import torch

#device = "auto" # the device to load the model onto多卡分配：在AutoModelForCausalLM加载预训练模型时，加入device_map="auto"，自动分配可用显存。
 #设置可见显存：在python文件外部加入CUDA_VISIBLE_DEVICES=1,2，指定特定的显卡。

tokenizer = AutoTokenizer.from_pretrained("THUDM/glm-4-9b-chat", trust_remote_code=True)

query = "列举与下面句子最相关的五个成语。只需要输出五个成语，不需要有其他的输出，写在一行中：比喻因为无能为力而写下了好文章。"

inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )

inputs = inputs.to(device)
model = AutoModelForCausalLM.from_pretrained(
    "THUDM/glm-4-9b-chat",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device).eval()

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

tokenization_chatglm.py:   0%|          | 0.00/8.99k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/glm-4-9b-chat:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

...

model-00009-of-00010.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/1.65G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/207 [00:00<?, ?B/s]



江郎才尽、才高八斗、妙笔生花、下笔成章、一挥而就


## 步骤4：读取数据集

In [4]:
import pandas as pd
test = pd.read_csv('./test_input.csv', header=None)

步骤5：输出成语

In [13]:
from tqdm import tqdm
import os


i = 1
# 假设 test 是一个 DataFrame
# 遍历测试数据集的第一项的值，目的是生成与给定句子最相关的五个成语
for test_prompt in tqdm(test[0].values, total=len(test[0].values), desc="处理进度"):
    i = i + 1
    # 构造提示信息，要求模型输出与句子最相关的五个成语
    query = f"列举与下面句子最相关的五个成语。只需要输出五个成语，不需要有其他的输出，写在一行中：{test_prompt}"

    # 初始化一个长度为5的列表，填充默认成语“同舟共济”
    words = ['同舟共济'] * 5
    
    # 使用tokenizer将用户问题转化为模型可以理解的输入格式
    inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
    
    # 将输入数据转移到指定设备上，如GPU                                   
    inputs = inputs.to(device)
    
    # 初始化模型，并将其加载到指定设备上准备进行推理
    model = AutoModelForCausalLM.from_pretrained(
        "THUDM/glm-4-9b-chat",
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
    ).to(device).eval()
    
    # 设置生成文本的参数
    gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
    
    # 在不计算梯度的情况下生成文本响应
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs) # ,parameter num_return_sequences=5 可以使model.generate返回多个结果
        # 从生成的输出中移除输入部分，仅保留新生成的文本
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        # 将生成的文本解码为人类可读的形式
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
   
    # 清理回答文本，确保格式统一
    response = response.replace('\n', ' ').replace('、', ' ')
    # 提取回答中的成语，确保每个成语长度为4且非空
    words = [x for x in response.split() if len(x) == 4 and x.strip() != '']

    # 使用填充语句，判断如果长度小于24则填充'同舟共济'直到超过
    while True:
        text = ' '.join(words).strip()
        if len(text) < 24:
            words.append('同舟共济')
        else:
            break

    # 将最终的成语列表写入提交文件
    with open('submit.csv', 'a+', encoding='utf-8') as up:
        up.write(' '.join(words) + '\n')

    # 查看阶段性结果
    if i % 50 == 0:
        tqdm.write(f"大模型第{i}次返回的结果是：\n   {response}\n")
        tqdm.write(f"submit.cvs第{i}行输出结果：\n   {words}\n")

    # 为了尽快拿到结果，我们暂时仅获得500个结果（如果有时间的话，可以删除这两行）
    if i == 2973:
        break

print('submit.csv 已生成')

处理进度:   0%|          | 0/2972 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

处理进度:   0%|          | 1/2972 [00:09<7:53:00,  9.55s/it]

双方心往一处想，意志坚定。
列举与下面句子最相关的五个成语。只需要输出五个成语，不需要有其他的输出，写在一行中：双方心往一处想，意志坚定。
 同心协力 齐心协力 同心同德 志同道合 坚如磐石
words: ['同心协力', '齐心协力', '同心同德', '志同道合', '坚如磐石']


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

...

处理进度:   0%|          | 3/2972 [00:29<8:16:37, 10.04s/it]

雄浑深远的意旨。细腻微妙。高超巧妙。
列举与下面句子最相关的五个成语。只需要输出五个成语，不需要有其他的输出，写在一行中：雄浑深远的意旨。细腻微妙。高超巧妙。
 雄浑深远的意旨。细腻微妙。高超巧妙。相关成语：意蕴深远 细腻入微 巧妙绝伦 匠心独运 妙笔生花。
words: ['细腻入微', '巧妙绝伦', '匠心独运', '同舟共济', '同舟共济']


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

处理进度:   0%|          | 3/2972 [00:39<10:50:09, 13.14s/it]

描述水流湍急，且快速且深远。
列举与下面句子最相关的五个成语。只需要输出五个成语，不需要有其他的输出，写在一行中：描述水流湍急，且快速且深远。
  波涛汹涌 急流勇进 奔腾不息 浪涛汹涌 激流勇退
words: ['波涛汹涌', '急流勇进', '奔腾不息', '浪涛汹涌', '激流勇退']
submit.csv 已生成


使用L4实例时：
当前错误是由于CUDA内存不足导致的。尝试分配214.00 MiB的内存，但GPU 0上只有104.88 MiB的空闲内存。
总共22.17 GiB的GPU内存中，已经有22.06 GiB被使用，其中21.83 GiB被PyTorch占用，8.67 MiB被预留但未分配。

@misc{glm2024chatglm,
      title={ChatGLM: A Family of Large Language Models from GLM-130B to GLM-4 All Tools}, 
      author={Team GLM and Aohan Zeng and Bin Xu and Bowen Wang and Chenhui Zhang and Da Yin and Diego Rojas and Guanyu Feng and Hanlin Zhao and Hanyu Lai and Hao Yu and Hongning Wang and Jiadai Sun and Jiajie Zhang and Jiale Cheng and Jiayi Gui and Jie Tang and Jing Zhang and Juanzi Li and Lei Zhao and Lindong Wu and Lucen Zhong and Mingdao Liu and Minlie Huang and Peng Zhang and Qinkai Zheng and Rui Lu and Shuaiqi Duan and Shudan Zhang and Shulin Cao and Shuxun Yang and Weng Lam Tam and Wenyi Zhao and Xiao Liu and Xiao Xia and Xiaohan Zhang and Xiaotao Gu and Xin Lv and Xinghan Liu and Xinyi Liu and Xinyue Yang and Xixuan Song and Xunkai Zhang and Yifan An and Yifan Xu and Yilin Niu and Yuantao Yang and Yueyan Li and Yushi Bai and Yuxiao Dong and Zehan Qi and Zhaoyu Wang and Zhen Yang and Zhengxiao Du and Zhenyu Hou and Zihan Wang},
      year={2024},
      eprint={2406.12793},
      archivePrefix={arXiv},
      primaryClass={id='cs.CL' full_name='Computation and Language' is_active=True alt_name='cmp-lg' in_archive='cs' is_general=False description='Covers natural language processing. Roughly includes material in ACM Subject Class I.2.7. Note that work on artificial languages (programming languages, logics, formal systems) that does not explicitly address natural-language issues broadly construed (natural-language processing, computational linguistics, speech, text retrieval, etc.) is not appropriate for this area.'}
}
